In [4]:
import corus
from tg.grammar_ru.common import Loc
import subprocess

In [5]:
import os
PATH = Loc.temp_path/'lenta.gz'

if not os.path.isfile(PATH):
    subprocess.call([
        'wget',
        'https://github.com/yutkin/Lenta.Ru-News-Dataset/releases/download/v1.0/lenta-ru-news.csv.gz',
        '-O',
        PATH
    ])

In [6]:
from yo_fluq_ds import *

def get_lenta():
    return Queryable(corus.load_lenta(PATH), 739351)

get_lenta().first()

LentaRecord(
    url='https://lenta.ru/news/2018/12/14/cancer/',
    title='Названы регионы России с\xa0самой высокой смертностью от\xa0рака',
    text='Вице-премьер по социальным вопросам Татьяна Голикова рассказала, в каких регионах России зафиксирована наиболее высокая смертность от рака, сообщает РИА Новости. По словам Голиковой, чаще всего онкологические заболевания становились причиной смерти в Псковской, Тверской, Тульской и Орловской областях, а также в Севастополе. Вице-премьер напомнила, что главные факторы смертности в России — рак и болезни системы кровообращения. В начале года стало известно, что смертность от онкологических заболеваний среди россиян снизилась впервые за три года. По данным Росстата, в 2017 году от рака умерли 289 тысяч человек. Это на 3,5 процента меньше, чем годом ранее.',
    topic='Россия',
    tags='Общество',
    date=None
)

In [7]:
import shutil

PROCESSED = Loc.processed_path/'lenta'
shutil.rmtree(PROCESSED, ignore_errors = True)
os.makedirs(PROCESSED)

buffer = []
length = 0
file_index = 0

for rec in get_lenta().feed(fluq.with_progress_bar()):
    buffer.append(rec.text)
    length+=len(rec.text)
    if length>1000000:
        text = f'# Partition {file_index}\n\n'
        text += '\n\n'.join(buffer)
        FileIO.write_text(text, PROCESSED/f'{file_index}.md')
        file_index+=1
        buffer = []
        length = 0

  0%|          | 0/739351 [00:00<?, ?it/s]

In [9]:
from tg.grammar_ru.corpus import CorpusBuilder

CorpusBuilder.convert_interformat_folder_to_corpus(
    Loc.corpus_path/'lenta.base.zip',
    PROCESSED,
    ['partition']
)

  0%|          | 0/986 [00:00<?, ?it/s]